In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
# https://stackoverflow.com/questions/32485907/matplotlib-and-numpy-create-a-calendar-heatmap
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

def main():
    dates, data = generate_data()
    fig, ax = plt.subplots(figsize=(6, 10))
    calendar_heatmap(ax, dates, data)
    plt.show()

def generate_data():
    num = 100
    data = np.random.randint(0, 20, num)
    start = dt.datetime(2015, 3, 13)
    dates = [start + dt.timedelta(days=i) for i in range(num)]
    return dates, data

def calendar_array(dates, data):
    i, j = zip(*[d.isocalendar()[1:] for d in dates])
    i = np.array(i) - min(i)
    j = np.array(j) - 1
    ni = max(i) + 1

    calendar = np.nan * np.zeros((ni, 7))
    calendar[i, j] = data
    return i, j, calendar


def calendar_heatmap(ax, dates, data):
    i, j, calendar = calendar_array(dates, data)
    im = ax.imshow(calendar, interpolation='none', cmap='hsv')
    label_days(ax, dates, i, j, calendar)
    label_months(ax, dates, i, j, calendar)
    ax.figure.colorbar(im)

def label_days(ax, dates, i, j, calendar):
#     ni, nj = calendar.shape
#     day_of_month = np.nan * np.zeros((ni, 7))
#     day_of_month[i, j] = [d.day for d in dates]

#     for (i, j), day in np.ndenumerate(day_of_month):
#         if np.isfinite(day):
#             ax.text(j, i, int(day), ha='center', va='center')

    ax.set(xticks=np.arange(7), 
           xticklabels=['M', 'T', 'W', 'R', 'F', 'S', 'S'])
    ax.xaxis.tick_top()

def label_months(ax, dates, i, j, calendar):
    month_labels = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
                             'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    months = np.array([d.month for d in dates])
    uniq_months = sorted(set(months))
    yticks = [i[months == m].mean() for m in uniq_months]
    labels = [month_labels[m - 1] for m in uniq_months]
    ax.set(yticks=yticks)
    ax.set_yticklabels(labels, rotation=90)

# main()

In [ ]:
cid = open('cid.txt', 'r').readline()
secret = open('secret.txt', 'r').readline()

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
username_dict = {"jimmyjhickey": "Jimmy",
                 "12101252392": "Lukas",
                 "aeolus9": "Ross",
                 "wdiedrick243": "Will",
                 "redtranscendent": "Michael",
                 "pboelter0705": "Paul",
                 "gabemcb": "Gabe"}

In [ ]:
shared_urls = ["https://open.spotify.com/playlist/23P7MTXWrqZkaqUAzegqvI", # gathers no moss
               "https://open.spotify.com/playlist/1u5KPMOFnOvWRlxyW13VFD", # chill 90s hiphop
               "https://open.spotify.com/playlist/1pfolyLLz4pQ3jLRYQk7d5", # under the covers
               "https://open.spotify.com/playlist/1SQjekEC7UpaFPHzidKQTd", # songs with stawry
               "https://open.spotify.com/playlist/3rWCGvc62Ea50aPvdj9iZe", # it's possible they're large
               "https://open.spotify.com/playlist/30Is1VfjAKbU1iqv4gFgoV" # jimmy
              ]

In [ ]:
data = {'Username': [],
        'Name': [],
        'Datetime': [],
        'Name-Date': []
       }
  
# Convert the dictionary into DataFrame
df = pd.DataFrame(data)


In [ ]:
for uri in shared_urls:
    playlist_URI = uri.split("/")[-1].split("?")[0]
    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    
    for i in range(len(sp.playlist_tracks(playlist_URI)["items"])):
        
        username = sp.playlist_tracks(playlist_URI)["items"][i]["added_by"]["id"]
        name = username_dict[username]
        
        date_time = sp.playlist_tracks(playlist_URI)["items"][i]["added_at"]

        
        df = df.append({'Username' : username,
                        'Name': name,
                        'Datetime': date_time,
                        'Name-Date': f"{name}-{date_time.split('T')[0]}"},
                         ignore_index = True)

In [ ]:
summary_df = df['Name-Date'].value_counts()
summary_df = summary_df.to_frame()
summary_df = summary_df.rename(columns = {"Name-Date": "Count"})
summary_df.index.name = 'Name-Date'
summary_df.reset_index(inplace=True)

In [ ]:
date_format = '%Y-%m-%d'

summary_df['Name'] = [ name_date.split('-')[0] for name_date in summary_df['Name-Date']]
summary_df['Date'] = [ dt.datetime.strptime(f"{name_date.split('-')[1]}-{name_date.split('-')[2]}-{name_date.split('-')[3]}", date_format)
                         for name_date in summary_df['Name-Date']]

In [ ]:
def make_plot(name): 
    total = sum(summary_df[summary_df['Name'] == name]['Count'])
    fig, ax = plt.subplots(figsize=(6, 10))
    calendar_heatmap(ax,
                     summary_df[summary_df['Name'] == name]['Date'],
                     summary_df[summary_df['Name'] == name]['Count'])
    fig.suptitle(f"{name} Playlist Contributions\nTotal={total}")
    plt.savefig(f'{name}_contributions_plot.png')



In [ ]:
make_plot("Jimmy")
make_plot("Ross")
make_plot("Lukas")
make_plot("Michael")
make_plot("Will")